In [1]:
import pandas
import matplotlib.pyplot

import plotly.plotly as py
import plotly.tools as tls
import plotly.graph_objs as go
from datetime import datetime
from copy import deepcopy
import json

In [2]:
def sum_of_item(group, item):
    return sum([q for (date, q) in group[item]])

def top_items_by_sales(k, group_name, group_sales = None):
    if group_sales is None:
        group_sales = json.load(open(group_name + '.json'))
    
    group_sales_total = [[sum_of_item(group_sales, item), item] for item in group_sales]
    group_sales_total.sort(reverse=True)
    group_sales_total = group_sales_total[:min(k, len(group_sales))]
    return [item for (sum_, item) in group_sales_total]  

In [3]:
top_items_by_sales(5, 'Яйцо')

['3186418', '3203662', '3319498', '3186437', '3243285']

In [4]:
def sales_share(item, other_items, group_name, group_sales = None):
    if group_sales is None:
        group_sales = json.load(open(group_name + '.json'))
    share = []
    for i in range(len(group_sales[item])):
        (date, a) = group_sales[item][i]
        b = 0.01
        for other_item in other_items:
            (date, a_) = group_sales[other_item][i]
            b += a_
        share.append((date, a / b))
    return share

In [5]:
best_eggs = top_items_by_sales(5, 'Яйцо')

In [6]:
def plot_shares(items, group_name, group_sales = None, graph_name=None):
    if group_sales is None:
        group_sales = json.load(open(group_name + '.json'))
    if graph_name is None:
        graph_name = group_name
    start_date = datetime(2015, 1, 3)
    end_date = datetime(2018, 1, 3)
    rng = pandas.date_range(start = start_date, end=end_date, freq='D')
    
    for item in items:
        share = sales_share(item, items, group_name, group_sales)
        ts = pandas.Series([q for (date, q) in share], index=rng)
        ts.plot(label=item)
    
    fig = matplotlib.pyplot.gcf()
    plotly_fig = tls.mpl_to_plotly( fig)
    plotly_fig['layout']['width'] = 1500
    plotly_fig['layout']['height'] = 1000
    py.plot(plotly_fig, filename=graph_name)

In [7]:
import numpy as np

def plot_week_shares(items, group_name, group_sales = None, graph_name=None):
    if group_sales is None:
        group_sales = json.load(open(group_name + '.json'))
    if graph_name is None:
        graph_name = group_name
    start_date = datetime(2015, 1, 3)
    end_date = datetime(2018, 1, 3)
    rng = pandas.date_range(start = start_date, end=end_date, freq='W')
    
    for item in items:
        share = sales_share(item, items, group_name, group_sales)
        share2 = [q for (date, q) in share]
        share3 = []
        for i in range(0, len(share2), 7):
            share3.append(np.average(share2[i: min(i + 7, len(share2))]))
        ts = pandas.Series(share3, index=rng)
        ts.plot(label=item)
    
    fig = matplotlib.pyplot.gcf()
    plotly_fig = tls.mpl_to_plotly( fig)
    plotly_fig['layout']['width'] = 1500
    plotly_fig['layout']['height'] = 1000
    py.plot(plotly_fig, filename=graph_name)
    

In [10]:
def plot2(name):
    plot_shares(top_items_by_sales(5, name), name, graph_name=name + '_по_дням')
    plot_week_shares(top_items_by_sales(5, name), name, graph_name=name + '_по_неделям')

In [21]:
plot2('Уход за телом')